# Pre-Process MSSIS AIS

Pre-processing of the Bermuda Case Study MSSIS vessel history dateset provided by Volpe from CSV to GeoJSON and GPKG.

In [ ]:
file = '../data/raw/volpe-bermuda-mssis-vessel-history.csv'

In [ ]:
out_path = '../data/interim/'

## volpe-bermuda-mssis-vessel-history.csv

In [ ]:
import pandas as pd
import geopandas
import json
import fiona

from shapely.geometry import Point

In [ ]:
!head '-n 5' $file

In [ ]:
df = pd.read_csv(file, sep=',',)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().any()

In [ ]:
df.EEZ.unique()

UN/LOCODE, the United Nations Code for Trade and Transport Locations assigns 'XZ' for international waters.

https://unece.org/trade/cefact/unlocode-code-list-country-and-territory

In [ ]:
df.EEZ.fillna('XZ', inplace=True)

In [ ]:
df.PortIndex.unique()

In [ ]:
df.AO.unique()

In [ ]:
df.SourceNum.unique()

In [ ]:
df.RxStnId.unique()

In [ ]:
df.drop(columns=['SourceNum', 'RxStnId'], inplace=True)

In [ ]:
df['TimeOfFix'] = pd.to_datetime(df['TimeOfFix'], unit='s')

In [ ]:
df.head()

In [ ]:
df[['MMSI', 'PortIndex']] = df[['MMSI','PortIndex']].astype('uint32')

In [ ]:
df[['AO']] = df[['AO']].astype('uint8')

In [ ]:
df.info()

In [ ]:
geometry = geopandas.points_from_xy(df.Longitude, df.Latitude)

In [ ]:
df.drop(columns=['Longitude', 'Latitude'], inplace=True)

In [ ]:
df.rename(columns = {'MMSI': 'vessel_mmsi', 'TimeOfFix': 'ais_time', 'SOG': 'ais_sog', 'Heading': 'ais_heading', 'PortIndex': 'mssis_wpi', 'EEZ': 'mssis_eez', 'AO': 'mssis_ao'}, inplace=True)

In [ ]:
df.info()

In [ ]:
gdf = geopandas.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

In [ ]:
gdf.head()

In [ ]:
gdf = gdf.rename(columns = {'geometry': 'ais_geometry'}).set_geometry('ais_geometry')

In [ ]:
gdf.info()

In [ ]:
gdf.crs

In [ ]:
gdf.head()

In [ ]:
mssis_ais_records_location_count = gdf.groupby(['mssis_wpi','mssis_eez', 'mssis_ao']).size().reset_index().rename(columns={0:'ais_count'}).to_dict(orient='records')

In [ ]:
len(mssis_ais_records_location_count)

In [ ]:
len(gdf.mssis_wpi.unique().tolist())

In [ ]:
mssis_ais_records_location_count

In [ ]:
gdf.groupby(['mssis_wpi','mssis_eez', 'mssis_ao']).size().reset_index().rename(columns={0:'ais_count'}).to_json(out_path + 'mssis-ais-region_v1.json', orient='records')

In [ ]:
gdf.to_file(out_path + 'mssis-ais-records_v1.geojson', driver='GeoJSON')

In [ ]:
gdf.to_file(out_path + 'mssis-ais-records_v1.gpkg', driver='GPKG')